# Calculate PPD distribution from data

In [35]:
import Pkg

# activate parent environment
Pkg.activate(normpath(joinpath(@__DIR__, ".")))
Pkg.resolve()
Pkg.instantiate()
Pkg.status()

using CSV
using DataFrames
using CairoMakie
using StatsBase

  Activating project at `c:\Users\lange\OneDrive\Projects\2024\EEM25\Congestion-Management-based-on-Thermal-Comfort`
  No Changes to `C:\Users\lange\OneDrive\Projects\2024\EEM25\Congestion-Management-based-on-Thermal-Comfort\Project.toml`
  No Changes to `C:\Users\lange\OneDrive\Projects\2024\EEM25\Congestion-Management-based-on-Thermal-Comfort\Manifest.toml`


Status `C:\Users\lange\OneDrive\Projects\2024\EEM25\Congestion-Management-based-on-Thermal-Comfort\Project.toml`
  [336ed68f] CSV v0.10.15
  [13f3f980] CairoMakie v0.13.1
  [a93c6f00] DataFrames v1.7.0
  [682c06a0] JSON v0.21.4
  [4076af6c] JuMP v1.23.6
⌃ [b8f27783] MathOptInterface v1.35.1
  [2d871509] OptHP v1.0.0-DEV `OptHP`
  [295af30f] Revise v3.7.2
  [2913bbd2] StatsBase v0.34.4
  [f269a46b] TimeZones v1.20.0
  [ade2ca70] Dates
Info Packages marked with ⌃ have new versions available and may be upgradable.


## Read data

In [46]:
df = CSV.read("data/PPD/PPD_ta_3001.csv", DataFrame)
dropmissing!(df)
select!(df, Not(:Column1))
first(df, 5)

Row,Ta,PPD
,Int64,Float64
1,15,35.9
2,15,38.0
3,15,25.3
4,15,51.8
5,15,5.3


## Calculate interquartiles ranges

In [38]:
# summarystats(df[!, :PPD])

In [53]:
dfg = groupby(df, :Ta)

# compute quantiles
dfg = combine(dfg) do df
    # filter outliers 3 std
    df = filter(:PPD => <(3 * std(df.PPD)), df)

    # Ti min max Q1 Q2 Q3 
    (
        min = quantile(df.PPD, max(0.0,0.25 - 1.5 * iqr(df.PPD))),
        max = quantile(df.PPD, min(1.0,0.75 + 1.5 * iqr(df.PPD))),
        Q1 = quantile(df.PPD, 0.25), 
        Q2 = quantile(df.PPD, 0.50),
        Q3 = quantile(df.PPD, 0.75)
    )
end

# save to CSV
CSV.write("data/PPD/PPD_quantiles.csv", dfg)

"data/PPD/PPD_quantiles.csv"